#CIS Week 10 Homework

**Instructor:** Lyle Ungar

**Content Creators:** Sanjeevini Ganni, Brittany Shields, Alessandra Rossi

In [8]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = 'jabrantl' #@param {type:"string"}
my_pod = 'superfluous-lyrebird' #@param ['Select', 'euclidean-wombat', 'sublime-newt', 'buoyant-unicorn', 'lackadaisical-manatee','indelible-stingray','superfluous-lyrebird','discreet-reindeer','quizzical-goldfish','ubiquitous-cheetah','nonchalant-crocodile','fashionable-lemur','spiffy-eagle','electric-emu','quotidian-lion','astute-jellyfish', 'quantum-herring']

# start timing
import time
try:t0;
except NameError: t0 = time.time()

We **strongly** recommend that you keep a separate document offline with your answers, and paste them in when you're ready to submit. Colab may reset and clear your notebook after a period of inactivity.

Go to Runtime -> Change runtime type -> Set Hardware Accelerator to "GPU"

###**Learning Objectives**
Finetune a Transformer model

How might word embedding perpetuate systemic bias?

How might we work with policymakers and companies to reduce the spread of fake news?

## Part I: Finetune GPT-2





GPT2 is a large-scale transformer-based language model developed by OpenAI. It is pre-trained on a large corpus of dataset of 8 million web pages has around 1.5 billion parameters. GPT2 is useful for language generation tasks, that is it predicts the next word, given some text.  

In this excersice we will be finetuning DistillGPT-2, distilled version of GPT2. We want the language model to generate wikipedia style of text. Hence, we will finetune distillGPT on [wikitext-2-raw-v1](https://huggingface.co/datasets/wikitext) dataset scrapped from wikipedia articles.

In [ ]:
#@title Install
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.0MB 7.6MB/s 
     |████████████████████████████████| 870kB 45.6MB/s 
     |████████████████████████████████| 3.2MB 51.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=a3e615193b7551af2fa30c7d07bc583f685c41ef12840b9b5ee168db83e5fc22
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 194kB 9.0MB/s 
     |████████████████████████████████| 245kB 13.3MB/s 
     |████████████████████████████████| 112kB 17.9MB/s 


In [ ]:
#@title Imports
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments

In [ ]:
#@title Load Datset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91. Subsequent calls will reuse this data.


Example of Dataset:

In [ ]:
datasets["train"][48]

{'text': " Two manga adaptations were produced , following each of the game 's main female protagonists Imca and Riela . They were Senjō no Valkyria 3 : Namo naki Chikai no Hana ( 戦場のヴァルキュリア3 名もなき誓いの花 , lit . Valkyria of the Battlefield 3 : The Flower of the Nameless Oath ) , illustrated by Naoyuki Fujisawa and eventually released in two volumes after being serialized in Dengeki Maoh between 2011 and 2012 ; and Senjō no Valkyria 3 : -Akaki Unmei no Ikusa Otome- ( 戦場のヴァルキュリア3 -赤き運命の戦乙女- , lit . Valkyria of the Battlefield 3 -The Valkyrie of the Crimson Fate ) , illustrated by Mizuki Tsuge and eventually released in a single volume by Kadokawa Shoten in 2012 . \n"}

####Tokenizer

In [ ]:
#Define the model Checkpoint
model_checkpoint = "distilgpt2"

#Tokenizer  
#To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. 
#This is all done by the AutoTokenizer class:  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

#We define a function to call the tokenizer on our texts:
def tokenize_function(examples):
    return tokenizer(examples["text"])

#Apply tokenizer to all the splits in our datasets object and drop the text coolumn
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
#we need to concatenate all our texts together then split the result in small chunks of a certain block_size
#Define the block size for training
block_size = 128

#preprocessing function that will group our texts
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

###Load pre-trained model

In [ ]:
#####Pretrained model
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

###Run Generations before Finetuning

In [ ]:
def run_generations(PROMPT_TEXT):
  device = model.device
  input_ids = tokenizer.encode(PROMPT_TEXT, return_tensors="pt").to(device)

  sample_output = model.generate(
      input_ids, 
      do_sample=True, 
      max_length=100, 
      top_p=0.95,
      num_return_sequences=3, 
      early_stopping=True
  )


  for i, output in enumerate(sample_output):
    print("Generation: "+ str(i) +"\n" + 100 * '-')
    print(tokenizer.decode(output, skip_special_tokens=True))
    print()


########
PROMPT_TEXT = 'Fast and the Furious '
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Fast and the Furious ___________________.
As for how much money could you make from the Furious?
I know not everybody is willing to bet on this, so here it is, the full game is available for free.
Advertisements

Generation: 1
----------------------------------------------------------------------------------------------------
Fast and the Furious!!! I’ve been watching this video for a lot of time and was wondering about what could possibly happen if he were cast in The Flash. I’m looking into that.
Now we have the details for The Flash to be revealed and I’ll give you the exact moment that I know what happened to the cast of the film and to get an idea about the direction the show is going in.
What I mean, this may be in the

Generation: 2
----------------------------------------------------------------------------------------------------
Fast and the Furious   was the tit

In [ ]:
############## Run Generation for different prompt texts of your choice
# PROMPT_TEXT = "The Astros won the World Series in 2017"
# run_generations(PROMPT_TEXT)


PROMPT_TEXT = 'Donald John Trump won the '
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Donald John Trump won the   election in 1980 by over 60%, and the only Republican in the history of the United States to win the presidency in 1980. With this result, we will be re-electing Barack Obama with some optimism, some hopeful, some pessimistic, and some hopeful, and some pessimistic. In fact, Donald Trump's victory is a clear indication of what is really going on in the world today. We will have to wait for the end of his presidency, and in

Generation: 1
----------------------------------------------------------------------------------------------------
Donald John Trump won the vernacular vote of his own voters. That means he'll win a landslide that will lead to a new Trump presidency, a Republican presidency, and a re-election that will see fewer of the two leading candidates being taken seriously.

















































Generation: 2

###Fine-Tuning (This will take around 20-25 minutes)

In [ ]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=2,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,3.758700,3.670479,15.653300,123.361000
2,3.668000,3.654829,15.685100,123.111000


TrainOutput(global_step=4668, training_loss=3.7530213448361853, metrics={'train_runtime': 1046.8924, 'train_samples_per_second': 4.459, 'total_flos': 2348513500594176.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 370306, 'init_mem_gpu_alloc_delta': 334744576, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 899782, 'train_mem_gpu_alloc_delta': 995278336, 'train_mem_cpu_peaked_delta': 155918522, 'train_mem_gpu_peaked_delta': 1766941696})

Perplexity of Validation Set

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 38.66


###Run Generations after Finetuning
Run generations for different prompts and compare them with the initial generations.

In [ ]:
PROMPT_TEXT = 'Fast and the Furious '
run_generations(PROMPT_TEXT)

############## Run Generation for different prompt texts of your choice
PROMPT_TEXT = 'The Astros won the World Series in 2017 '
run_generations(PROMPT_TEXT)


PROMPT_TEXT = 'Donald John Trump '
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Fast and the Furious 
 " Black " and " Gold " ", respectively, were both released together in the United States. They were released in the United Kingdom, Spain, Austria, Croatia, Serbia, Switzerland, Turkey, Spain, Switzerland, Ukraine, Vietnam, and Thailand during World War II, and they were released in the United States. 
 The first two albums, Black's, and Gold, were released jointly by J.P. Lewis, David W. Anderson, and

Generation: 1
----------------------------------------------------------------------------------------------------
Fast and the Furious 
 was also released as the second and third sequel in the series. The original two sequel titles ( one for the original series ) were released as the third and fourth sequel installments, and as part of their respective " prequel series " were also released as the third and fourth sequels. 
 = = History = = 
 The orig

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
The Astros won the World Series in 2017 
 Astros pitcher Michael Jackson is considered a favorite to win the All @-@ American League MVP award. He is widely considered the best offensive player in baseball, with three All @-@ MLB starters ranked second among all @-@ time greatest relievers. He finished the 2016 season with 18 wins and a record of 33 – 18. " Michael Jackson is a three @-@ time All @-@ time MVP award for the season, tied for

Generation: 1
----------------------------------------------------------------------------------------------------
The Astros won the World Series in 2017 
 = = = Major League Baseball 
 = = = Opening day ( MLB ) = = = = 
 Beginning in July 2013, the Astros won the National League West, West Division Series, West Series and West Division titles to complete seven seasons. The Phillies won the World Series in 2003 and 2008. They finished 

In [9]:
#@markdown Question 0: How do the generations after finetuning compare to the generations before finetuning. Do you think finetuning acheived its purpose of generating Wikipedia like texts? 
q0 = "The tone and feel of the text does read like a Wikipedia article. However, the actual facts being presented do not make much sense and the actual sentences are not super clear. " #@param{type:"string"}

try:t0;
except NameError: t0 = time.time()

## Part II: Primer on Gender Bias in Word Embeddings

Read the following primer that describes the societal implications of gender bias in word embeddings: \\
“Man is to Doctor as Woman is to Nurse: The Gender Bias of Word Embeddings: Why we should worry about gender inequality in Natural Language Processing techniques” \\
https://towardsdatascience.com/gender-bias-word-embeddings-76d9806a0e17 \\
*9 min read*, Tommaso Buonocore, 2019


In [2]:
#@markdown Question 1: Identify a compelling quote or example from the article that resonated with you.  How does it emphasize the societal responsibility of data scientists?
q1 = "'Even scarier results can be retrieved switching the domain of interest from occupations to adjectives. According to the embeddings, men are cocky, decent, crafty, brilliant, clever and humble. Women instead--unsurprisingly at this point--are described as sassy, sexy, tasteful, attractive and gorgeous.' - This quote is quite telling of what the underlying embedding of the gender terms is. The one about occupations at least shows *some* examples that are not just part of the traditionally female stereotype. However, the one for adjectives does not give many substantive attributes to female, other than superficial ones about looks or those having to do with the home. I can see how this would occur if a corpus of older literature is used for training.  "#@param{type:"string"}

try:t1;
except NameError: t1 = time.time()

##Part III: Tech Paper on Gender Bias in Word Embeddings

In the previous article, the author referenced that data scientists are introducing new strategies to reduce biases in word embeddings.  In 2016, researchers from Boston University and Microsoft Research published a paper with significant findings demonstrating the prevalence of gender bias in word embeddings, as well as proposed mitigation measures.  A 5-minute summary of the technical components of the paper can be found [here](https://towardsdatascience.com/tackling-gender-bias-in-word-embeddings-c965f4076a10) and the original paper can be found [here](https://arxiv.org/abs/1607.06520). \

“Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings” \
https://arxiv.org/abs/1607.06520 \
*30 min read*, 2016





In [3]:
#@markdown Question 2: In the “Discussion” section, the authors consider the responsibility of machine language programmers with regards to bias in word embeddings.  Describe their stance.
q2 = "While we may opt to blame society for making ML models biased (based on training), we should work to de-bias our data in ML to not amplify existing biases. " #@param{type:"string"}

#@markdown Question 3: How could word embeddings amplify societal stereotypes?
q3 = "In a job application, for example, attributes about the candidate may be aligned with biases that are inherited from the corpus of literature on which it was trained. This may lead to the issue of not choosing a qualified candidate because of their race, gender, etc." #@param{type:"string"}

#@markdown Question 4: How could the authors’ proposed technological interventions address this concern?
q4 = "By reducing gender associations with words, future ML applications can remove biases in the embeddings. This would help to create diversity in the workplace. The advantage is that instead of biasing the application process to specifically include underrepresented individuals, we can improve diversity without explicit bias in any way. This leads towards the overarching goal of fairness and equality. " #@param{type:"string"}

try:t2;
except NameError: t2 = time.time()

##Part IV: Fake News

The following article provides an overview of neural fake news, including potential defenses against it as well as suggestions for further research.

“An Exhaustive Guide to Detecting and Fighting Neural Fake News using NLP” \
•	https://www.analyticsvidhya.com/blog/2019/12/detect-fight-neural-fake-news-nlp/ \
•	*16-minute read*, Analytics Vidhya, 2019

In 2019, Jack Clark, the Policy Director at OpenAI, gave testimony to the US House Intelligence Committee.  You can either read his written 10-page testimony here or watch his live testimony here (jump to minute 11:00 to watch his 5 minute testimony).

Written Testimony of Jack Clark, Policy Director, OpenAI \
•	https://intelligence.house.gov/uploadedfiles/clark_deepfakes_sfr.pdf \
•	or \
•	https://www.c-span.org/video/?461679-1/house-intelligence-committee-hearing-deepfake-videos \
*Jump to minute 11:00* \
•	House Permanent Select Committee on Intelligence, 2019 \


In [4]:
#@markdown Question 5: Jack Clark enumerated several ideas for interventions.  Discuss one of the specific ideas he suggested.
q5 = "Institutional interventions. He suggests tech companies developing and sharing tools for identifying fake media." #@param{type:"string"}

#@markdown Question 6: How might technologists work with policymakers and companies to prevent the spread of fake news?
q6 = "First thing is to develop new methods for detecting face news. This is often best done by making fake news first and then detecting other fake news with the same tools (as is done with Grover). This requires funding and policy makers should allocate these funds to researchers who will work on this problem" #@param{type:"string"}

#@markdown Question 7: •	OpenAI is not making GPT-3 publicly available, claiming that they are concerned that it might be used for doing bad. Do you think that was a good decision? Why?
q7 = "I understand their concern, but w/o public inspection of the models, issues of bias and harm may go undetected. Or they may be observed, but the specific reason may be difficult to determine. " #@param{type:"string"}

try:t3;
except NameError: t3 = time.time()

## PART V: Know your Pod

With two other members of your pod, do the following. Have each of them recommend you their favorite song. Then, listen to those songs and write down your thoughts for each. (~100 words each)

In [5]:
know_a_pod_1 = "" #@param{type:"string"}
know_a_pod_2 = "" #@param{type:"string"}

try:t4;
except NameError: t4 = time.time()

#Submission

Once you're done, click on 'Share' and add the link to the box below.

In [6]:
link = 'https://colab.research.google.com/drive/1lozqySjTUcxQREJbsvhZfQ2lNJUcLS8x?usp=sharing' #@param {type:"string"}

In [10]:
import time
import numpy as np
import urllib.parse
from IPython import display
from IPython.display import IFrame


t5 = time.time()

#@markdown #Run Cell to Show Airtable Form
#@markdown ##**Confirm your answers and then click "Submit"**


def prefill_form(src, fields: dict):
  '''
  src: the original src url to embed the form
  fields: a dictionary of field:value pairs,
  e.g. {"pennkey": my_pennkey, "location": my_location}
  '''
  prefill_fields = {}
  for key in fields:
      new_key = 'prefill_' + key
      prefill_fields[new_key] = fields[key]
  prefills = urllib.parse.urlencode(prefill_fields)
  src = src + prefills
  return src



#autofill fields if they are not present
#a missing pennkey and pod will result in an Airtable warning
#which is easily fixed user-side.
try: my_pennkey;
except NameError: my_pennkey = ""

try: my_pod;
except NameError: my_pod = "Select"

try: q0;
except NameError: q0 = ""

try: q1;
except NameError: q1 = ""

try: q2;
except NameError: q2 = ""

try: q3;
except NameError: q3 = ""

try: q4;
except NameError: q4 = ""

try: q5;
except NameError: q5 = ""

try: q6;
except NameError: q6 = ""

try: q7;
except NameError: q7 = ""

try: know_a_pod_1;
except NameError: know_a_pod_1 = ""

try: know_a_pod_2;
except NameError: know_a_pod_2 = ""

try: link;
except NameError: link = ""

fields = {"pennkey": my_pennkey,
          "pod": my_pod,
          "q0": q0,
          "q1": q1,
          "q2": q2,
          "q3": q3,
          "q4": q4,
          "q5": q5,
          "q6": q6,
          "q7": q7,
          "know_a_pod_1": know_a_pod_1,
          "know_a_pod_2": know_a_pod_2,
          "link": link}

src = "https://airtable.com/embed/shrgundiXUEJFVzwd?"


#now instead of the original source url, we do: src = prefill_form(src, fields)
display.display(IFrame(src = prefill_form(src, fields), width = 800, height = 400))